# Cosyne 2019 NWB:N Tutorial - Extracellular Electrophysiology

## Set up NWB file
NWB files require a session start time to be entered with a timezone field.

In [1]:
from pynwb import NWBFile
from datetime import datetime
from dateutil import tz

start_time = datetime(2018, 4, 25, 2, 30, 3, tzinfo=tz.gettz('US/Pacific'))

nwbfile = NWBFile(identifier='Mouse5_Day3',
                 session_description='mouse in open exploration and theta maze',  # required
                 session_start_time=start_time,  # required
                 experimenter='My Name',  # optional
                 session_id='session_id',  # optional
                 institution='University of My Institution',  # optional
                 lab='My Lab Name',  # optional
                 related_publications='DOI:10.1016/j.neuron.2016.12.011')  # optional

## Subject info

In [2]:
from pynwb.file import Subject

nwbfile.subject = Subject(age='9 months', description='mouse 5',
                          species='Mus musculus', sex='M')

## Position
The `Position` object is a `MultiContainerInterface` that holds one or more `SpatialSeries` objects, which are a subclass of `TimeSeries`. Here, we put a `SpatialSeries` object called `'position'` in a `Position` object, and put that in a `ProcessingModule` named `'behavior'`.
<img src="images/position.png" width="800">

In [3]:
import numpy as np
from pynwb.behavior import SpatialSeries, Position

position_data = np.array([
    np.linspace(0, 10, 100),
    np.linspace(1, 8, 100)]).T
spatial_series_object = SpatialSeries(
   name='position', data=position_data,
   reference_frame='unknown',
   conversion=1.0, resolution=np.nan,
   timestamps=np.linspace(0, 100) / 200)

In [4]:
pos_obj = Position(spatial_series=spatial_series_object)
behavior_module = nwbfile.create_processing_module(
    name='behavior',
    description='data relevant to behavior')

behavior_module.add_data_interface(pos_obj)

## Write to file

In [5]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('test_ephys.nwb', 'w') as io:
    io.write(nwbfile)

## Trials
Trials is another `DynamicTable` that lives an `/intervals/trials`.

In [ ]:
nwbfile.add_trial_column('correct', description='correct trial')
nwbfile.add_trial(start_time=1.0, stop_time=5.0, correct=True)
nwbfile.add_trial(start_time=6.0, stop_time=10.0, correct=False)

## Electrodes table
Extracellular electrodes are stored in a `electrodes`, which is a `DynamicTable`. `electrodes` has several required fields: x, y, z, impedance, location, filtering, and electrode_group. Here, we also demonstrate how to add optional columns to a table by adding the `'label'` column.<img src="images/electrodes_table.png" width="300">

In [6]:
nwbfile.add_electrode_column('label', 'label of electrode')
shank_channels = [4, 3]

electrode_counter = 0
device = nwbfile.create_device('implant')
for shankn, nelecs in enumerate(shank_channels):
    electrode_group = nwbfile.create_electrode_group(
       name='shank{}'.format(shankn),
       description='electrode group for shank {}'.format(shankn),
       device=device,
       location='brain area')
    for ielec in range(nelecs):
        nwbfile.add_electrode(
           x=5.3, y=1.5, z=8.5, imp=np.nan,
           location='unknown', filtering='unknown',
           group=electrode_group,
           label='shank{}elec{}'.format(shankn, ielec))
        electrode_counter += 1

all_table_region = nwbfile.create_electrode_table_region(
  list(range(electrode_counter)), 'all electrodes')

## LFP
`LFP` is another `MultiContainerInterface`. It holds one or more `ElectricalSeries` objects, which are `TimeSeries`. Here, we put an `ElectricalSeries` named `'lfp'` in an `LFP` object, in a `ProcessingModule` named `'ecephys'`.
<img src="images/lfp.png" width="800">

In [7]:
from pynwb.ecephys import ElectricalSeries, LFP
lfp_data = np.random.randn(100, 7)
ecephys_module = nwbfile.create_processing_module(
    name='ecephys',
    description='extracellular electrophysiology data')
ecephys_module.add_data_interface(
LFP(ElectricalSeries('lfp', lfp_data, all_table_region, 
rate=1000., resolution=.001, conversion=1.)))

## Spike Times
Spike times are stored in another `DynamicTable` of subtype `Units`. The main `Units` table is at `/units` in the HDF5 file. You can add columns to the `Units` table just like you did for `electrodes`.

In [8]:
for shankn, channels in enumerate(shank_channels):
    for n_units_per_shank in range(np.random.poisson(lam=5)):
        n_spikes = np.random.poisson(lam=10)
        spike_times = np.abs(np.random.randn(n_spikes))
        nwbfile.add_unit(spike_times=spike_times)

## Write and read
Data arrays are read passively from the file. That means `TimeSeries.data` does not read the entire data object, but presents an h5py object that can be indexed to read data. Index this array just like a numpy array to read only a specific section of the array, or use the `[:]` operator to read the entire thing.

In [10]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('test_ephys.nwb', 'w') as io:
    io.write(nwbfile)

with NWBHDF5IO('test_ephys.nwb', 'r') as io:
    nwbfile2 = io.read()

    print(nwbfile2.modules['ecephys']['LFP'].electrical_series['lfp'].data[:])

[[-1.18748994e+00 -1.02725896e+00  6.80845832e-01  1.41010819e+00
   9.93847913e-01 -1.07257974e+00 -1.76200432e+00]
 [-1.91578338e+00 -9.00207419e-01  5.91149951e-01 -2.58572290e+00
  -1.73342216e+00  1.15397889e+00 -7.66208021e-01]
 [-2.28052884e-01 -2.08688104e-01 -1.40442801e-01  2.51521720e-01
   6.04263260e-02 -9.98354434e-02  1.08457621e+00]
 [-8.30306082e-01  2.78702874e-02 -1.15557359e+00 -1.53361656e+00
  -8.10439648e-01  8.64130938e-02 -1.01399851e+00]
 [ 9.88800813e-01  8.60196534e-01 -1.13070345e+00  4.21478483e-01
  -1.39917114e+00 -3.02864657e-01  1.43830828e+00]
 [ 4.58407372e-02  1.55692901e-01 -9.87254173e-01  9.20414330e-01
  -1.05719026e+00 -1.58959302e+00  1.01406388e+00]
 [ 1.02244064e+00  4.42150202e-01  1.57886662e-01 -1.47574098e+00
  -5.73636339e-01  1.72859034e+00  3.17727796e-01]
 [ 4.40830850e-03  2.42902334e-02  3.14347080e-02  8.60116748e-01
   5.63744801e-02  8.70175600e-01 -6.54398389e-01]
 [-2.70055784e-01  2.80836658e-01  2.96627813e-01 -1.22255795e+0

## Accessing data regions
You can easily read subsections of datasets

In [11]:
io = NWBHDF5IO('test_ephys.nwb', 'r')
nwbfile2 = io.read()

print('section of lfp:')
print(nwbfile2.modules['ecephys']['LFP'].electrical_series['lfp'].data[:10,:5])
print('')
print('')
print('spike times from first unit:')
print(nwbfile2.units['spike_times'][0])
io.close()

section of lfp:
[[-1.18748994 -1.02725896  0.68084583  1.41010819  0.99384791]
 [-1.91578338 -0.90020742  0.59114995 -2.5857229  -1.73342216]
 [-0.22805288 -0.2086881  -0.1404428   0.25152172  0.06042633]
 [-0.83030608  0.02787029 -1.15557359 -1.53361656 -0.81043965]
 [ 0.98880081  0.86019653 -1.13070345  0.42147848 -1.39917114]
 [ 0.04584074  0.1556929  -0.98725417  0.92041433 -1.05719026]
 [ 1.02244064  0.4421502   0.15788666 -1.47574098 -0.57363634]
 [ 0.00440831  0.02429023  0.03143471  0.86011675  0.05637448]
 [-0.27005578  0.28083666  0.29662781 -1.22255795 -2.49816636]
 [-2.19238143  1.46674518 -0.49098211 -0.13401105 -2.15160877]]


spike times from first unit:
[0.02647075 0.07243559 1.50147784 0.18226922 1.16359468 1.4911805
 0.25810137 0.58790677 0.5408381 ]
